### Automação do SUAP-IFPE versão 0.1

#### Importando webdriver e abrindo SUAP

In [2]:
import pandas as pd

arquivo = 'priori_0_100.xlsx'
tombos_df = pd.read_excel(arquivo)
#display(tombos_df.head())
print(tombos_df.info())

autentica = pd.read_excel('ls.xlsx')

login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

indisponivel = "indisponivel"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   tombo   91 non-null     int64
dtypes: int64(1)
memory usage: 856.0 bytes
None


In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time



def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [4]:
def trocar_sala():
    
    abrir_suap()
    
    for linha in tombos_df["tombo"]:
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")

            valor_sala_suap = nav.find_element_by_xpath('//*[@id="id_sala"]').get_attribute('value')
            valor_nova_sala = '2100'

            # time.sleep(1)

            if valor_sala_suap != valor_nova_sala:
                nova_sala = Select(nav.find_element_by_id('id_sala'))
                nova_sala.select_by_value(valor_nova_sala)
                time.sleep(1)
                nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[2]/input').click()
        
        except NoSuchElementException:
            continue

    
    nav.quit()

In [5]:
def listar_carga():
    
    abrir_suap()
    
    tombo_impressao = []
    carga_impressao = []
    descricao_impressao = []    
    
    for linha in tombos_df["tombo"]:
        
        tombo_impressao.append(linha)
        # print(linha)
        
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")

            carga = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[7]/td[2]").get_attribute('innerText')
            carga_impressao.append(carga)
            # print(carga)

            descricao = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td[2]/span[1]").get_attribute('innerText')
            descricao_impressao.append(descricao)
            # print(descricao)
        except NoSuchElementException:
            carga_impressao.append(indisponivel)
            descricao_impressao.append(indisponivel)
            continue
            
    tabela_carga = pd.DataFrame()
    tabela_carga['tombo'] = tombo_impressao
    tabela_carga['descricao'] = descricao_impressao
    tabela_carga['carga'] = carga_impressao

    #display(tabela_carga)

    tabela_carga.to_excel("lista_carga"+arquivo, index=False, sheet_name="plan1")
        
    nav.quit()

In [6]:
def impressao_etiqueta():
    
    abrir_suap()
    
    tombo_etiqueta = []    
    empenho_impressao = []
    uasg_impressao = []
    nf_impressao = []
    dtnf_impressao = []
    qrcode_impressao = []
    
    for linha in tombos_df["tombo"]:
        
        tombo_etiqueta.append(linha)
        # print(linha)
        
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")
            nav.find_element_by_link_text("Detalhes da Entrada").click()

            empenho = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[12]/td/a").get_attribute('innerText')
            empenho = empenho[:12]
            empenho_impressao.append(empenho)
            # print(empenho)
        
            uasg = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[1]/td").get_attribute('innerText')
            uasg_impressao.append(uasg)
            # print(uasg)

            nf = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td").get_attribute('innerText')
            nf_impressao.append(nf)
            # print(nf)

            data = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[5]/td").get_attribute('innerText')
            dtnf_impressao.append(data)
            # print(data)

            qrcode = str(linha)+" "+empenho+" "+" "+uasg+" "+nf+" "+data
            qrcode_impressao.append(qrcode)

            
        except NoSuchElementException:
            # print(indisponivel)
            empenho_impressao.append(indisponivel)
            uasg_impressao.append(indisponivel)
            nf_impressao.append(indisponivel)
            dtnf_impressao.append(indisponivel)
            qrcode_impressao.append(indisponivel)
            continue    
        
    tabela_impressao = pd.DataFrame()
    tabela_impressao['patrimonio'] = tombo_etiqueta
    tabela_impressao['empenho'] = empenho_impressao
    tabela_impressao['uasg'] = uasg_impressao
    tabela_impressao['nota fiscal'] = nf_impressao
    tabela_impressao['data nota fiscal'] = dtnf_impressao
    tabela_impressao['qrcode'] = qrcode_impressao
    
#     display(tabela_impressao)
    
    tabela_impressao.to_excel("impressao_"+arquivo, index=False, sheet_name="plan1")
        

    nav.quit()

In [7]:
impressao_etiqueta()
# trocar_sala()
# listar_carga()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.159)
